In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

In [2]:
llm = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./.files/chapter3_1984.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = FAISS.from_documents(docs, cache_embeddings)
retriever = vectorstore.as_retriever()
memory = ConversationBufferMemory()

In [3]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]


prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant. Answer questions using only the following context and {history}. If you don't "
         "know the answer just say you don't know, don't make it up:\n\n{context}"),
        ("human", "{question}"),
    ]
)

chain = {"context": retriever, "question": RunnablePassthrough(),
         "history": RunnableLambda(load_memory)} | prompt | llm


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question}, {"output": result.content}
    )
    print(result)


In [4]:
invoke_chain("Is Aaronson guilty?")

content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [5]:
invoke_chain("What message did he write in the table?")

content='He wrote "2+2=5" in the dust on the table.'


In [6]:
invoke_chain("Who is Julia?")

content='Julia is a character mentioned in the text.'


In [7]:
invoke_chain("What was my first question?")

content='Your first question was "Is Aaronson guilty?"'
